# STAT29000 Project 10 Examples

In this project we are working with `SQLAlchemy`, in Python, to interact with an SQLite database. 

You can find a set of examples [here](https://datamine.purdue.edu/seminars/spring2020/stat19000project09examples.pdf) or on scholar:

`/class/datamine/data/spring2020/stat19000project09examples.pdf`.

In this notebook, I will run through the same set of queries using SQLAlchemy. I will demonstrate how to run direct queries using SQL syntax, as well as the ORM way of doing things.

In addition, you can find more SQL examples (including joins) [here](https://datamine.purdue.edu/seminars/spring2020/stat19000project10examples.pdf) or on scholar:

`/class/datamine/data/spring2020/stat19000project10examples.pdf`.

In addition to what we've provided here, please take the time to look through [this](https://www.fullstackpython.com/object-relational-mappers-orms.html). At first, looking at the SQLAlchemy code, it may not be immediately apparent why this is useful. However, if you look at some examples using the [Django ORM](https://docs.djangoproject.com/en/3.0/topics/db/queries/), it may be more clear how an ORM may help abstract certain concepts away and simplify things for people unfamiliar with SQL, but familiar with Python.

In [3]:
import sqlalchemy as db

In [5]:
# First, we must establish a connection with the database.
engine = db.create_engine("sqlite:////class/datamine/data/spring2020/chinook.db")

# Then, to run direct, SQL queries:
with engine.connect() as con:
    result = con.execute("SELECT * FROM employees;")
    for r in result:
        print(r)
    
# Or, use the ORM
employees_table = db.Table('employees', db.MetaData(), autoload=True, autoload_with=engine)

# Form the query
query = db.select([employees_table])

# Execute query
with engine.connect() as con:
    result = con.execute(query)
    for r in result:
        print(r)
        
# Note that the with ____ as ___ syntax is nothing to be afraid of.
# Without going into too much detail, it basically ensures that something is
# done at the start and end of the code block. In this case, it ensures that
# the connection to the database is automatically closed after the block is
# over with. So instead of:
connection = engine.connect()
result = connection.execute("SELECT * FROM employees;")
print(result)
connection.close()

# You can do:
with engine.connect() as con:
    result = con.execute("SELECT * FROM employees;")
    print(result)
    
# Although it isn't necessarily easier, using this notation helps ensure
# you don't accidentally forget to close the connection.

(1, 'Adams', 'Andrew', 'General Manager', None, '1962-02-18 00:00:00', '2002-08-14 00:00:00', '11120 Jasper Ave NW', 'Edmonton', 'AB', 'Canada', 'T5K 2N1', '+1 (780) 428-9482', '+1 (780) 428-3457', 'andrew@chinookcorp.com')
(2, 'Edwards', 'Nancy', 'Sales Manager', 1, '1958-12-08 00:00:00', '2002-05-01 00:00:00', '825 8 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 2T3', '+1 (403) 262-3443', '+1 (403) 262-3322', 'nancy@chinookcorp.com')
(3, 'Peacock', 'Jane', 'Sales Support Agent', 2, '1973-08-29 00:00:00', '2002-04-01 00:00:00', '1111 6 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 5M5', '+1 (403) 262-3443', '+1 (403) 262-6712', 'jane@chinookcorp.com')
(4, 'Park', 'Margaret', 'Sales Support Agent', 2, '1947-09-19 00:00:00', '2003-05-03 00:00:00', '683 10 Street SW', 'Calgary', 'AB', 'Canada', 'T2P 5G3', '+1 (403) 263-4423', '+1 (403) 263-4289', 'margaret@chinookcorp.com')
(5, 'Johnson', 'Steve', 'Sales Support Agent', 2, '1965-03-03 00:00:00', '2003-10-17 00:00:00', '7727B 41 Ave', 'Calgary', 'A

NameError: name 'connection' is not defined

Let's start running through some examples shall we?

In [6]:
metadata = db.MetaData()
metadata.reflect(bind=engine)

# What are the tables in the database?
print(metadata.tables.keys())

# or
inspector = db.engine.reflection.Inspector.from_engine(engine)
inspector.get_table_names()

dict_keys(['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'invoices', 'tracks', 'media_types', 'playlist_track', 'playlists', 'sqlite_sequence', 'sqlite_stat1'])


['albums',
 'artists',
 'customers',
 'employees',
 'genres',
 'invoice_items',
 'invoices',
 'media_types',
 'playlist_track',
 'playlists',
 'sqlite_sequence',
 'sqlite_stat1',
 'tracks']

You can think of a database index just like the index of a text book. Without the index the way to find something is to scan page by page. This is slow. The index allows us to look for a topic alphabetically and turn directly to the corresponding page. At a very high level, a database index is a way to sacrifice space for speed. We need to add a good chunk of pages to the book, but now we can find things much faster. 

In [7]:
# Are there any indices?
for table in inspector.get_table_names():
    for index in inspector.get_indexes(table):
        print(index)

{'name': 'IFK_AlbumArtistId', 'column_names': ['ArtistId'], 'unique': 0}
{'name': 'IFK_CustomerSupportRepId', 'column_names': ['SupportRepId'], 'unique': 0}
{'name': 'IFK_EmployeeReportsTo', 'column_names': ['ReportsTo'], 'unique': 0}
{'name': 'IFK_InvoiceLineTrackId', 'column_names': ['TrackId'], 'unique': 0}
{'name': 'IFK_InvoiceLineInvoiceId', 'column_names': ['InvoiceId'], 'unique': 0}
{'name': 'IFK_InvoiceCustomerId', 'column_names': ['CustomerId'], 'unique': 0}
{'name': 'IFK_PlaylistTrackTrackId', 'column_names': ['TrackId'], 'unique': 0}
{'name': 'IFK_TrackMediaTypeId', 'column_names': ['MediaTypeId'], 'unique': 0}
{'name': 'IFK_TrackGenreId', 'column_names': ['GenreId'], 'unique': 0}
{'name': 'IFK_TrackAlbumId', 'column_names': ['AlbumId'], 'unique': 0}


In [8]:
# select all of the columns for the first employee only
with engine.connect() as con:
    result = con.execute("SELECT * FROM employees LIMIT(1);")
    for row in result:
        print(row)
        
# or 
with engine.connect() as con:
    query = db.select([metadata.tables["employees"]]).limit(1)
    result = con.execute(query)
    for row in result:
        print(row)

(1, 'Adams', 'Andrew', 'General Manager', None, '1962-02-18 00:00:00', '2002-08-14 00:00:00', '11120 Jasper Ave NW', 'Edmonton', 'AB', 'Canada', 'T5K 2N1', '+1 (780) 428-9482', '+1 (780) 428-3457', 'andrew@chinookcorp.com')
(1, 'Adams', 'Andrew', 'General Manager', None, datetime.datetime(1962, 2, 18, 0, 0), datetime.datetime(2002, 8, 14, 0, 0), '11120 Jasper Ave NW', 'Edmonton', 'AB', 'Canada', 'T5K 2N1', '+1 (780) 428-9482', '+1 (780) 428-3457', 'andrew@chinookcorp.com')


In [9]:
# How do you get the column names of a table?
print(metadata.tables["employees"].columns)
print(metadata.tables["employees"].c)

# select the first and last name for the first employee only
with engine.connect() as con:
    result = con.execute("SELECT FirstName, LastName FROM employees LIMIT(1);")
    for row in result:
        print(row)
        
# or 
with engine.connect() as con:
    query = db.select([metadata.tables["employees"].c.FirstName, metadata.tables["employees"].c.LastName]).limit(1)
    result = con.execute(query)
    for row in result:
        print(row)

['employees.EmployeeId', 'employees.LastName', 'employees.FirstName', 'employees.Title', 'employees.ReportsTo', 'employees.BirthDate', 'employees.HireDate', 'employees.Address', 'employees.City', 'employees.State', 'employees.Country', 'employees.PostalCode', 'employees.Phone', 'employees.Fax', 'employees.Email']
['employees.EmployeeId', 'employees.LastName', 'employees.FirstName', 'employees.Title', 'employees.ReportsTo', 'employees.BirthDate', 'employees.HireDate', 'employees.Address', 'employees.City', 'employees.State', 'employees.Country', 'employees.PostalCode', 'employees.Phone', 'employees.Fax', 'employees.Email']
('Andrew', 'Adams')
('Andrew', 'Adams')


At this point in time, you may notice that knowing SQL syntax can definitely be beneficial. The SQL version is shorter and arguably easier to read. With that being said, the ORM can make it easy for those who are familiar with Python but not familiar with SQL.

In [10]:
# Let's make things look nicer:
Albums = metadata.tables["albums"]
Artists = metadata.tables["artists"]
Customers = metadata.tables["customers"]
Employees = metadata.tables["employees"]
Genres = metadata.tables["genres"]
InvoiceItems = metadata.tables["invoice_items"]
Invoices = metadata.tables["invoices"]
MediaTypes = metadata.tables["media_types"]
Playlists = metadata.tables["playlists"]
PlaylistTrack = metadata.tables["playlist_track"]

In [13]:
# select all of the first and last name for the first employee only
with engine.connect() as con:
    result = con.execute("SELECT FirstName, LastName FROM employees LIMIT(1);")
    for row in result:
        print(row)
        
# or 
with engine.connect() as con:
    query = db.select([Employees.c.FirstName, Employees.c.LastName]).limit(1)
    result = con.execute(query)
    for row in result:
        print(row)

('Andrew', 'Adams')
('Andrew', 'Adams')


In [14]:
# Note that case doesn't matter for the pure SQL version, but it does for the ORM portion

# select all of the first and last name for the first employee
with engine.connect() as con:
    result = con.execute("select firstname, lastname from employees limit(1);")
    for row in result:
        print(row)

('Andrew', 'Adams')


In [15]:
with engine.connect() as con:
    result = con.execute("SELECT FirstName, LastName FROM employees LIMIT(2);")
    for row in result:
        print(row)
        
# or 
with engine.connect() as con:
    query = db.select([Employees.c.FirstName, Employees.c.LastName]).limit(2)
    result = con.execute(query)
    for row in result:
        print(row)

('Andrew', 'Adams')
('Nancy', 'Edwards')
('Andrew', 'Adams')
('Nancy', 'Edwards')


In [16]:
# select the first and last name for all of the employees
with engine.connect() as con:
    result = con.execute("SELECT FirstName, LastName FROM employees;")
    for row in result:
        print(row)
        
# or 
with engine.connect() as con:
    query = db.select([Employees.c.FirstName, Employees.c.LastName])
    result = con.execute(query)
    for row in result:
        print(row)

('Andrew', 'Adams')
('Nancy', 'Edwards')
('Jane', 'Peacock')
('Margaret', 'Park')
('Steve', 'Johnson')
('Michael', 'Mitchell')
('Robert', 'King')
('Laura', 'Callahan')
('Andrew', 'Adams')
('Nancy', 'Edwards')
('Jane', 'Peacock')
('Margaret', 'Park')
('Steve', 'Johnson')
('Michael', 'Mitchell')
('Robert', 'King')
('Laura', 'Callahan')


In [17]:
# Note that in SQL if you select the same column twice, it appears twice
with engine.connect() as con:
    result = con.execute("SELECT FirstName, Firstname, LastName FROM employees LIMIT(1);")
    for row in result:
        print(row)
        
# BUT, using the orm, it does not
with engine.connect() as con:
    query = db.select([Employees.c.FirstName, Employees.c.FirstName, Employees.c.LastName]).limit(1)
    result = con.execute(query)
    for row in result:
        print(row)

('Andrew', 'Andrew', 'Adams')
('Andrew', 'Adams')


In [19]:
# Let's get the first 2 names after the 5th name
with engine.connect() as con:
    result = con.execute("SELECT FirstName, LastName FROM employees LIMIT(2) OFFSET(5);")
    for row in result:
        print(row)
        
with engine.connect() as con:
    query = db.select([Employees.c.FirstName, Employees.c.LastName]).limit(2).offset(5)
    result = con.execute(query)
    for row in result:
        print(row)

('Michael', 'Mitchell')
('Robert', 'King')
('Michael', 'Mitchell')
('Robert', 'King')


In [20]:
# What if we wanted the last two names?
with engine.connect() as con:
    result = con.execute("SELECT FirstName, LastName FROM employees LIMIT(2) OFFSET(6);")
    for row in result:
        print(row)
        
with engine.connect() as con:
    query = db.select([Employees.c.FirstName, Employees.c.LastName]).limit(2).offset(6)
    result = con.execute(query)
    for row in result:
        print(row)
        
# But as soon as the database adds a new row, this won't work. 

('Robert', 'King')
('Laura', 'Callahan')
('Robert', 'King')
('Laura', 'Callahan')


In [24]:
# So how is the order currently determined?
with engine.connect() as con:
    result = con.execute("SELECT * FROM employees;")
    for row in result:
        print(row)
        
with engine.connect() as con:
    query = db.select([Employees])
    result = con.execute(query)
    for row in result:
        print(row)
        
# Order is determined by the first column, which is?
print(Employees.c)

# EmployeeId

(1, 'Adams', 'Andrew', 'General Manager', None, '1962-02-18 00:00:00', '2002-08-14 00:00:00', '11120 Jasper Ave NW', 'Edmonton', 'AB', 'Canada', 'T5K 2N1', '+1 (780) 428-9482', '+1 (780) 428-3457', 'andrew@chinookcorp.com')
(2, 'Edwards', 'Nancy', 'Sales Manager', 1, '1958-12-08 00:00:00', '2002-05-01 00:00:00', '825 8 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 2T3', '+1 (403) 262-3443', '+1 (403) 262-3322', 'nancy@chinookcorp.com')
(3, 'Peacock', 'Jane', 'Sales Support Agent', 2, '1973-08-29 00:00:00', '2002-04-01 00:00:00', '1111 6 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 5M5', '+1 (403) 262-3443', '+1 (403) 262-6712', 'jane@chinookcorp.com')
(4, 'Park', 'Margaret', 'Sales Support Agent', 2, '1947-09-19 00:00:00', '2003-05-03 00:00:00', '683 10 Street SW', 'Calgary', 'AB', 'Canada', 'T2P 5G3', '+1 (403) 263-4423', '+1 (403) 263-4289', 'margaret@chinookcorp.com')
(5, 'Johnson', 'Steve', 'Sales Support Agent', 2, '1965-03-03 00:00:00', '2003-10-17 00:00:00', '7727B 41 Ave', 'Calgary', 'A

In [28]:
# Let's dig into that more later.
# Another way to get the last two names, would then be to use DESC and ORDER BY
with engine.connect() as con:
    result = con.execute("SELECT * FROM employees ORDER BY EmployeeId DESC;")
    for row in result:
        print(row)
        
with engine.connect() as con:
    query = db.select([Employees]).order_by(db.desc("EmployeeId"))
    result = con.execute(query)
    for row in result:
        print(row)

(8, 'Callahan', 'Laura', 'IT Staff', 6, '1968-01-09 00:00:00', '2004-03-04 00:00:00', '923 7 ST NW', 'Lethbridge', 'AB', 'Canada', 'T1H 1Y8', '+1 (403) 467-3351', '+1 (403) 467-8772', 'laura@chinookcorp.com')
(7, 'King', 'Robert', 'IT Staff', 6, '1970-05-29 00:00:00', '2004-01-02 00:00:00', '590 Columbia Boulevard West', 'Lethbridge', 'AB', 'Canada', 'T1K 5N8', '+1 (403) 456-9986', '+1 (403) 456-8485', 'robert@chinookcorp.com')
(6, 'Mitchell', 'Michael', 'IT Manager', 1, '1973-07-01 00:00:00', '2003-10-17 00:00:00', '5827 Bowness Road NW', 'Calgary', 'AB', 'Canada', 'T3B 0C5', '+1 (403) 246-9887', '+1 (403) 246-9899', 'michael@chinookcorp.com')
(5, 'Johnson', 'Steve', 'Sales Support Agent', 2, '1965-03-03 00:00:00', '2003-10-17 00:00:00', '7727B 41 Ave', 'Calgary', 'AB', 'Canada', 'T3B 1Y7', '1 (780) 836-9987', '1 (780) 836-9543', 'steve@chinookcorp.com')
(4, 'Park', 'Margaret', 'Sales Support Agent', 2, '1947-09-19 00:00:00', '2003-05-03 00:00:00', '683 10 Street SW', 'Calgary', 'AB',

In [29]:
# Now we just need to add the limit back
with engine.connect() as con:
    result = con.execute("SELECT * FROM employees ORDER BY EmployeeId DESC LIMIT(2);")
    for row in result:
        print(row)
        
with engine.connect() as con:
    query = db.select([Employees]).order_by(db.desc("EmployeeId")).limit(2)
    result = con.execute(query)
    for row in result:
        print(row)

(8, 'Callahan', 'Laura', 'IT Staff', 6, '1968-01-09 00:00:00', '2004-03-04 00:00:00', '923 7 ST NW', 'Lethbridge', 'AB', 'Canada', 'T1H 1Y8', '+1 (403) 467-3351', '+1 (403) 467-8772', 'laura@chinookcorp.com')
(7, 'King', 'Robert', 'IT Staff', 6, '1970-05-29 00:00:00', '2004-01-02 00:00:00', '590 Columbia Boulevard West', 'Lethbridge', 'AB', 'Canada', 'T1K 5N8', '+1 (403) 456-9986', '+1 (403) 456-8485', 'robert@chinookcorp.com')
(8, 'Callahan', 'Laura', 'IT Staff', 6, datetime.datetime(1968, 1, 9, 0, 0), datetime.datetime(2004, 3, 4, 0, 0), '923 7 ST NW', 'Lethbridge', 'AB', 'Canada', 'T1H 1Y8', '+1 (403) 467-3351', '+1 (403) 467-8772', 'laura@chinookcorp.com')
(7, 'King', 'Robert', 'IT Staff', 6, datetime.datetime(1970, 5, 29, 0, 0), datetime.datetime(2004, 1, 2, 0, 0), '590 Columbia Boulevard West', 'Lethbridge', 'AB', 'Canada', 'T1K 5N8', '+1 (403) 456-9986', '+1 (403) 456-8485', 'robert@chinookcorp.com')


In [30]:
# ORDER BY goes in ascending order, but you can be explicit if you want to be
with engine.connect() as con:
    result = con.execute("SELECT * FROM employees ORDER BY EmployeeId ASC;")
    for row in result:
        print(row)
        
with engine.connect() as con:
    query = db.select([Employees]).order_by(db.asc("EmployeeId"))
    result = con.execute(query)
    for row in result:
        print(row)

(1, 'Adams', 'Andrew', 'General Manager', None, '1962-02-18 00:00:00', '2002-08-14 00:00:00', '11120 Jasper Ave NW', 'Edmonton', 'AB', 'Canada', 'T5K 2N1', '+1 (780) 428-9482', '+1 (780) 428-3457', 'andrew@chinookcorp.com')
(2, 'Edwards', 'Nancy', 'Sales Manager', 1, '1958-12-08 00:00:00', '2002-05-01 00:00:00', '825 8 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 2T3', '+1 (403) 262-3443', '+1 (403) 262-3322', 'nancy@chinookcorp.com')
(3, 'Peacock', 'Jane', 'Sales Support Agent', 2, '1973-08-29 00:00:00', '2002-04-01 00:00:00', '1111 6 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 5M5', '+1 (403) 262-3443', '+1 (403) 262-6712', 'jane@chinookcorp.com')
(4, 'Park', 'Margaret', 'Sales Support Agent', 2, '1947-09-19 00:00:00', '2003-05-03 00:00:00', '683 10 Street SW', 'Calgary', 'AB', 'Canada', 'T2P 5G3', '+1 (403) 263-4423', '+1 (403) 263-4289', 'margaret@chinookcorp.com')
(5, 'Johnson', 'Steve', 'Sales Support Agent', 2, '1965-03-03 00:00:00', '2003-10-17 00:00:00', '7727B 41 Ave', 'Calgary', 'A

In [31]:
# Let's look at the title column
with engine.connect() as con:
    result = con.execute("SELECT title FROM employees;")
    for row in result:
        print(row)
        
with engine.connect() as con:
    query = db.select([Employees.c.Title])
    result = con.execute(query)
    for row in result:
        print(row)

('General Manager',)
('Sales Manager',)
('Sales Support Agent',)
('Sales Support Agent',)
('Sales Support Agent',)
('IT Manager',)
('IT Staff',)
('IT Staff',)
('General Manager',)
('Sales Manager',)
('Sales Support Agent',)
('Sales Support Agent',)
('Sales Support Agent',)
('IT Manager',)
('IT Staff',)
('IT Staff',)


In [33]:
# The result is redundant. What if we just wanted a list of titles but there were 10000 employees!
# This is where SELECT DISTINCT comes in handy. SELECT DISTINCT returns only unique results or unique 
# combinations depending on what column(s) you SELECT DISTINCT
with engine.connect() as con:
    result = con.execute("SELECT DISTINCT title FROM employees;")
    for row in result:
        print(row)
        
with engine.connect() as con:
    query = db.select([Employees.c.Title]).distinct()
    result = con.execute(query)
    for row in result:
        print(row)

('General Manager',)
('Sales Manager',)
('Sales Support Agent',)
('IT Manager',)
('IT Staff',)
('General Manager',)
('Sales Manager',)
('Sales Support Agent',)
('IT Manager',)
('IT Staff',)


In [34]:
# But if you wanted firstname/title combos (of which all are unique), you'd get:
with engine.connect() as con:
    result = con.execute("SELECT DISTINCT title, firstname FROM employees;")
    for row in result:
        print(row)
        
with engine.connect() as con:
    query = db.select([Employees.c.Title, Employees.c.FirstName]).distinct()
    result = con.execute(query)
    for row in result:
        print(row)

('General Manager', 'Andrew')
('Sales Manager', 'Nancy')
('Sales Support Agent', 'Jane')
('Sales Support Agent', 'Margaret')
('Sales Support Agent', 'Steve')
('IT Manager', 'Michael')
('IT Staff', 'Robert')
('IT Staff', 'Laura')
('General Manager', 'Andrew')
('Sales Manager', 'Nancy')
('Sales Support Agent', 'Jane')
('Sales Support Agent', 'Margaret')
('Sales Support Agent', 'Steve')
('IT Manager', 'Michael')
('IT Staff', 'Robert')
('IT Staff', 'Laura')


In [36]:
# SQL also has some built in function including but not limited to: MIN, MAX, COUNT, SUM, AVG
with engine.connect() as con:
    result = con.execute("SELECT MIN(Total) FROM invoices;")
    for row in result:
        print(row)
        
with engine.connect() as con:
    query = db.select([db.sql.func.Min(Invoices.c.Total)])
    result = con.execute(query)
    for row in result:
        print(row)
        
# Note: Just ignore the warning, its unimportant for now.

(0.99,)
(Decimal('0.99'),)


/home/kamstut/.pyenv/versions/3.6.4/lib/python3.6/site-packages/sqlalchemy/sql/sqltypes.py:666: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  "storage." % (dialect.name, dialect.driver)


In [49]:
# We can also check out the schema of a table to see how it was built
# The schema info is stored in the metadata
metadata.tables
metadata.tables.get("albums").c.AlbumId.type

INTEGER()

In [50]:
with engine.connect() as con:
    result = con.execute("SELECT MAX(Total) FROM invoices;")
    for row in result:
        print(row)
        
with engine.connect() as con:
    query = db.select([db.sql.func.Max(Invoices.c.Total)])
    result = con.execute(query)
    for row in result:
        print(row)

(25.86,)
(Decimal('25.86'),)


In [51]:
with engine.connect() as con:
    result = con.execute("SELECT AVG(Total) FROM invoices;")
    for row in result:
        print(row)
        
with engine.connect() as con:
    query = db.select([db.sql.func.Avg(Invoices.c.Total)])
    result = con.execute(query)
    for row in result:
        print(row)

(5.651941747572825,)
(5.651941747572825,)


In [54]:
# Important note: LIMIT doesn't effect the functions. They will operate on the entire row
with engine.connect() as con:
    result = con.execute("SELECT AVG(Total) FROM invoices LIMIT(2);")
    for row in result:
        print(row)
        
with engine.connect() as con:
    query = db.select([db.sql.func.Avg(Invoices.c.Total)]).limit(2)
    result = con.execute(query)
    for row in result:
        print(row)

(5.651941747572825,)
(5.651941747572825,)


In [55]:
# Same behavior for all function...
with engine.connect() as con:
    result = con.execute("SELECT SUM(Total) FROM invoices LIMIT(2);")
    for row in result:
        print(row)
        
with engine.connect() as con:
    query = db.select([db.sql.func.Sum(Invoices.c.Total)]).limit(2)
    result = con.execute(query)
    for row in result:
        print(row)

(2328.600000000004,)
(Decimal('2328.60'),)


In [57]:
# Count is useful too. It counts the number of rows when COUNT(*), otherwise if it is COUNT(column_name), 
# it returns the count of non null values
with engine.connect() as con:
    result = con.execute("SELECT COUNT(*) FROM invoices;")
    for row in result:
        print(row)
        
with engine.connect() as con:
    query = db.select([db.sql.func.Count(Invoices)])
    result = con.execute(query)
    for row in result:
        print(row)

(412,)
(412,)


In [59]:
# What if we want to condition our queries? For example, we want only invoices for Brazil.
# This is where the WHERE clause comes in handy:
with engine.connect() as con:
    result = con.execute("SELECT * FROM invoices WHERE BillingCountry='Brazil';")
    for row in result:
        print(row)
        
with engine.connect() as con:
    query = db.select([Invoices]).where(Invoices.c.BillingCountry=="Brazil")
    result = con.execute(query)
    for row in result:
        print(row)

(25, 10, '2009-04-09 00:00:00', 'Rua Dr. Falcão Filho, 155', 'São Paulo', 'SP', 'Brazil', '01007-010', 8.91)
(34, 12, '2009-05-23 00:00:00', 'Praça Pio X, 119', 'Rio de Janeiro', 'RJ', 'Brazil', '20040-020', 0.99)
(35, 13, '2009-06-05 00:00:00', 'Qe 7 Bloco G', 'Brasília', 'DF', 'Brazil', '71020-677', 1.98)
(57, 11, '2009-09-06 00:00:00', 'Av. Paulista, 2022', 'São Paulo', 'SP', 'Brazil', '01310-200', 1.98)
(58, 13, '2009-09-07 00:00:00', 'Qe 7 Bloco G', 'Brasília', 'DF', 'Brazil', '71020-677', 3.96)
(68, 11, '2009-10-17 00:00:00', 'Av. Paulista, 2022', 'São Paulo', 'SP', 'Brazil', '01310-200', 13.86)
(80, 13, '2009-12-10 00:00:00', 'Qe 7 Bloco G', 'Brasília', 'DF', 'Brazil', '71020-677', 5.94)
(98, 1, '2010-03-11 00:00:00', 'Av. Brigadeiro Faria Lima, 2170', 'São José dos Campos', 'SP', 'Brazil', '12227-000', 3.98)
(121, 1, '2010-06-13 00:00:00', 'Av. Brigadeiro Faria Lima, 2170', 'São José dos Campos', 'SP', 'Brazil', '12227-000', 3.96)
(123, 11, '2010-06-17 00:00:00', 'Av. Paulista,

In [60]:
# And it can be combined with functions too, finding the total for Brazil invoices
with engine.connect() as con:
    result = con.execute("SELECT SUM(Total) FROM invoices WHERE BillingCountry='Brazil';")
    for row in result:
        print(row)
        
with engine.connect() as con:
    query = db.select([db.sql.func.Sum(Invoices.c.Total)]).where(Invoices.c.BillingCountry=="Brazil")
    result = con.execute(query)
    for row in result:
        print(row)

(190.09999999999997,)
(Decimal('190.10'),)


In [61]:
# BETWEEN can be useful as well, and selects the rows that are between two values
# The function is inclusive, which means if we want numbers between 1-10 we would include 1 and 10 in the list.

# Let's get invoices where the total is between 13 and 14
with engine.connect() as con:
    result = con.execute("SELECT * FROM invoices WHERE Total BETWEEN 13 and 14;")
    for row in result:
        print(row)
        
with engine.connect() as con:
    query = db.select([Invoices]).where(Invoices.c.Total.between(13, 14))
    result = con.execute(query)
    for row in result:
        print(row)

(5, 23, '2009-01-11 00:00:00', '69 Salem Street', 'Boston', 'MA', 'USA', '2113', 13.86)
(12, 2, '2009-02-11 00:00:00', 'Theodor-Heuss-Straße 34', 'Stuttgart', None, 'Germany', '70174', 13.86)
(19, 40, '2009-03-14 00:00:00', '8, Rue Hanovre', 'Paris', None, 'France', '75002', 13.86)
(26, 19, '2009-04-14 00:00:00', '1 Infinite Loop', 'Cupertino', 'CA', 'USA', '95014', 13.86)
(33, 57, '2009-05-15 00:00:00', 'Calle Lira, 198', 'Santiago', None, 'Chile', None, 13.86)
(40, 36, '2009-06-15 00:00:00', 'Tauentzienstraße 8', 'Berlin', None, 'Germany', '10789', 13.86)
(47, 15, '2009-07-16 00:00:00', '700 W Pender Street', 'Vancouver', 'BC', 'Canada', 'V6C 1G8', 13.86)
(54, 53, '2009-08-16 00:00:00', '113 Lupus St', 'London', None, 'United Kingdom', 'SW1V 3EN', 13.86)
(61, 32, '2009-09-16 00:00:00', '696 Osborne Street', 'Winnipeg', 'MB', 'Canada', 'R3L 2B9', 13.86)
(68, 11, '2009-10-17 00:00:00', 'Av. Paulista, 2022', 'São Paulo', 'SP', 'Brazil', '01310-200', 13.86)
(75, 49, '2009-11-17 00:00:00'

In [72]:
# Date example, get invoices between 1/1/2013 and 2/1/2013
with engine.connect() as con:
    result = con.execute("SELECT * FROM invoices WHERE InvoiceDate BETWEEN '2013-1-1' AND '2013-2-1';")
    for row in result:
        print(row)
        
with engine.connect() as con:
    query = db.select([Invoices]).where(Invoices.c.InvoiceDate.between('2013-1-1', '2013-2-1'))
    result = con.execute(query)
    for row in result:
        print(row)
        
# Oh no! This is incorrect. Why? We need to zero-pad the month and day

(392, 4, '2013-10-03 00:00:00', 'Ullevålsveien 14', 'Oslo', None, 'Norway', '0171', 1.98)
(393, 6, '2013-10-03 00:00:00', 'Rilská 3174/6', 'Prague', None, 'Czech Republic', '14300', 1.98)
(394, 8, '2013-10-04 00:00:00', 'Grétrystraat 63', 'Brussels', None, 'Belgium', '1000', 3.96)
(395, 12, '2013-10-05 00:00:00', 'Praça Pio X, 119', 'Rio de Janeiro', 'RJ', 'Brazil', '20040-020', 5.94)
(396, 18, '2013-10-08 00:00:00', '627 Broadway', 'New York', 'NY', 'USA', '10012-2612', 8.91)
(397, 27, '2013-10-13 00:00:00', '1033 N Park Ave', 'Tucson', 'AZ', 'USA', '85719', 13.86)
(398, 41, '2013-10-21 00:00:00', '11, Place Bellecour', 'Lyon', None, 'France', '69002', 0.99)
(399, 42, '2013-11-03 00:00:00', '9, Place Louis Barthou', 'Bordeaux', None, 'France', '33000', 1.98)
(400, 44, '2013-11-03 00:00:00', 'Porthaninkatu 9', 'Helsinki', None, 'Finland', '00530', 1.98)
(401, 46, '2013-11-04 00:00:00', '3 Chatham Street', 'Dublin', 'Dublin', 'Ireland', None, 3.96)
(402, 50, '2013-11-05 00:00:00', 'C/ S

In [73]:
with engine.connect() as con:
    result = con.execute("SELECT * FROM invoices WHERE InvoiceDate BETWEEN '2013-01-01' AND '2013-02-01';")
    for row in result:
        print(row)
        
with engine.connect() as con:
    query = db.select([Invoices]).where(Invoices.c.InvoiceDate.between('2013-01-01', '2013-02-01'))
    result = con.execute(query)
    for row in result:
        print(row)

(333, 30, '2013-01-02 00:00:00', '230 Elgin Street', 'Ottawa', 'ON', 'Canada', 'K2P 1L7', 8.91)
(334, 39, '2013-01-07 00:00:00', '4, Rue Milton', 'Paris', None, 'France', '75009', 13.86)
(335, 53, '2013-01-15 00:00:00', '113 Lupus St', 'London', None, 'United Kingdom', 'SW1V 3EN', 0.99)
(336, 54, '2013-01-28 00:00:00', '110 Raeburn Pl', 'Edinburgh ', None, 'United Kingdom', 'EH4 1HH', 1.98)
(337, 56, '2013-01-28 00:00:00', '307 Macacha Güemes', 'Buenos Aires', None, 'Argentina', '1106', 1.98)
(338, 58, '2013-01-29 00:00:00', '12,Community Centre', 'Delhi', None, 'India', '110017', 3.96)
(339, 3, '2013-01-30 00:00:00', '1498 rue Bélanger', 'Montréal', 'QC', 'Canada', 'H2G 1A7', 5.94)
(333, 30, datetime.datetime(2013, 1, 2, 0, 0), '230 Elgin Street', 'Ottawa', 'ON', 'Canada', 'K2P 1L7', Decimal('8.91'))
(334, 39, datetime.datetime(2013, 1, 7, 0, 0), '4, Rue Milton', 'Paris', None, 'France', '75009', Decimal('13.86'))
(335, 53, datetime.datetime(2013, 1, 15, 0, 0), '113 Lupus St', 'London

In [79]:
# Great! How many of those were in the UK?
with engine.connect() as con:
    result = con.execute("SELECT * FROM invoices WHERE InvoiceDate BETWEEN '2013-01-01' AND '2013-02-01' AND BillingCountry='United Kingdom';")
    for row in result:
        print(row)
        
with engine.connect() as con:
    query = db.select([Invoices]).where(db.and_(Invoices.c.InvoiceDate.between('2013-01-01', '2013-02-01'), Invoices.c.BillingCountry=="United Kingdom"))
    result = con.execute(query)
    for row in result:
        print(row)

(335, 53, '2013-01-15 00:00:00', '113 Lupus St', 'London', None, 'United Kingdom', 'SW1V 3EN', 0.99)
(336, 54, '2013-01-28 00:00:00', '110 Raeburn Pl', 'Edinburgh ', None, 'United Kingdom', 'EH4 1HH', 1.98)
(335, 53, datetime.datetime(2013, 1, 15, 0, 0), '113 Lupus St', 'London', None, 'United Kingdom', 'SW1V 3EN', Decimal('0.99'))
(336, 54, datetime.datetime(2013, 1, 28, 0, 0), '110 Raeburn Pl', 'Edinburgh ', None, 'United Kingdom', 'EH4 1HH', Decimal('1.98'))


In [83]:
# Excellent! Although not explicitly mentioned earlier, AND, OR, and NOT function just like normal
# logical operators in any language. You can use parenthesis to force order of operations as well
with engine.connect() as con:
    result = con.execute("SELECT * FROM invoices WHERE Total > 20 OR Total < 5 AND BillingCountry='Brazil';")
    for row in result:
        print(row)
        
with engine.connect() as con:
    query = db.select([Invoices]).where(db.and_(db.or_(Invoices.c.Total > 20, Invoices.c.Total < 5), (Invoices.c.BillingCountry=="Brazil")))
    result = con.execute(query)
    for row in result:
        print(row)

(34, 12, '2009-05-23 00:00:00', 'Praça Pio X, 119', 'Rio de Janeiro', 'RJ', 'Brazil', '20040-020', 0.99)
(35, 13, '2009-06-05 00:00:00', 'Qe 7 Bloco G', 'Brasília', 'DF', 'Brazil', '71020-677', 1.98)
(57, 11, '2009-09-06 00:00:00', 'Av. Paulista, 2022', 'São Paulo', 'SP', 'Brazil', '01310-200', 1.98)
(58, 13, '2009-09-07 00:00:00', 'Qe 7 Bloco G', 'Brasília', 'DF', 'Brazil', '71020-677', 3.96)
(96, 45, '2010-02-18 00:00:00', 'Erzsébet krt. 58.', 'Budapest', None, 'Hungary', 'H-1073', 21.86)
(98, 1, '2010-03-11 00:00:00', 'Av. Brigadeiro Faria Lima, 2170', 'São José dos Campos', 'SP', 'Brazil', '12227-000', 3.98)
(121, 1, '2010-06-13 00:00:00', 'Av. Brigadeiro Faria Lima, 2170', 'São José dos Campos', 'SP', 'Brazil', '12227-000', 3.96)
(132, 13, '2010-07-31 00:00:00', 'Qe 7 Bloco G', 'Brasília', 'DF', 'Brazil', '71020-677', 0.99)
(154, 10, '2010-11-14 00:00:00', 'Rua Dr. Falcão Filho, 155', 'São Paulo', 'SP', 'Brazil', '01007-010', 1.98)
(155, 12, '2010-11-14 00:00:00', 'Praça Pio X, 11

In [ ]:
# Uh oh! We wanted invoices where the Total was either greater than 20 or less than 5 and from Brazil. The 
# first example is incorrect because AND has precedence over OR (in the same way multiplication has 
# precedence over addition and subtraction). What that first statement was equivalent to is:
with engine.connect() as con:
    result = con.execute("SELECT * FROM invoices WHERE Total > 20 OR (Total < 5 AND BillingCountry='Brazil');")
    for row in result:
        print(row)

# But what we wanted was:
with engine.connect() as con:
    result = con.execute("SELECT * FROM invoices WHERE (Total > 20 OR Total < 5) AND BillingCountry='Brazil';")
    for row in result:
        print(row)
        
# Which fixes it!

# Notice that due to the way that the ORM handles db.and_ and db.or_, we were forced to use the correct
# order of operations!

In [102]:
# Now the last topic we will touch on is called aliasing (using AS). I'd recommend reading:
# https://www.w3schools.com/sql/sql_alias.asp

# You can alias a column
with engine.connect() as con:
    result = con.execute("SELECT BillingCountry AS Country FROM invoices LIMIT(1);")
    for row in result:
        for c, v in row.items():
            print(c, v)
        
with engine.connect() as con:
    country = Invoices.c.BillingCountry.label("Country")
    query = db.select([country]).limit(1)
    result = con.execute(query)

    for row in result:
        for c, v in row.items():
            print(c, v)
            
# Where normally it would be:
with engine.connect() as con:
    result = con.execute("SELECT BillingCountry FROM invoices LIMIT(1);")
    for row in result:
        for c, v in row.items():
            print(c, v)
        
with engine.connect() as con:
#     country = Invoices.c.BillingCountry.label("Country")
    query = db.select([Invoices.c.BillingCountry]).limit(1)
    result = con.execute(query)

    for row in result:
        for c, v in row.items():
            print(c, v)

Country Germany
Country Germany
BillingCountry Germany
BillingCountry Germany


In [103]:
# You can also alias multiple columns:
with engine.connect() as con:
    result = con.execute("SELECT BillingCountry AS Country, Total AS DollarValue FROM invoices LIMIT(1);")
    for row in result:
        for c, v in row.items():
            print(c, v)
        
with engine.connect() as con:    
    query = db.select([Invoices.c.BillingCountry.label("Country"), Invoices.c.Total.label("DollarValue")]).limit(1)
    result = con.execute(query)

    for row in result:
        for c, v in row.items():
            print(c, v)

Country Germany
DollarValue 1.98
Country Germany
DollarValue 1.98


In [111]:
# You can alias a table too
with engine.connect() as con:
    result = con.execute("SELECT inv.BillingCountry FROM invoices AS inv LIMIT(1);")
    for row in result:
        for c, v in row.items():
            print(c, v)
        
with engine.connect() as con:
    inv = Invoices.alias("inv")
    
    query = db.select([inv.c.BillingCountry]).limit(1)
    result = con.execute(query)

    for row in result:
        for c, v in row.items():
            print(c, v)
            
print(str(query)) # As you can see the table is aliased

BillingCountry Germany
BillingCountry Germany
SELECT inv."BillingCountry" 
FROM invoices AS inv
 LIMIT :param_1
SELECT inv."BillingCountry" 
FROM invoices AS inv
 LIMIT ? OFFSET ?


In [118]:
# You can query multiple tables
with engine.connect() as con:
    result = con.execute("SELECT i.BillingCountry, c.FirstName, c.LastName FROM invoices AS i, customers AS c WHERE i.CustomerId = c.CustomerId LIMIT(10);")          
    for row in result:
        print(row)
        
with engine.connect() as con:
    i = Invoices.alias("i")
    c = Customers.alias("c")
    
    query = db.select([i.c.BillingCountry, c.c.FirstName, c.c.LastName]).where(i.c.CustomerId == c.c.CustomerId).limit(10)
    result = con.execute(query)

    for row in result:
        print(row)
        
print(str(query))

('Brazil', 'Luís', 'Gonçalves')
('Brazil', 'Luís', 'Gonçalves')
('Brazil', 'Luís', 'Gonçalves')
('Brazil', 'Luís', 'Gonçalves')
('Brazil', 'Luís', 'Gonçalves')
('Brazil', 'Luís', 'Gonçalves')
('Brazil', 'Luís', 'Gonçalves')
('Germany', 'Leonie', 'Köhler')
('Germany', 'Leonie', 'Köhler')
('Germany', 'Leonie', 'Köhler')
('Brazil', 'Luís', 'Gonçalves')
('Brazil', 'Luís', 'Gonçalves')
('Brazil', 'Luís', 'Gonçalves')
('Brazil', 'Luís', 'Gonçalves')
('Brazil', 'Luís', 'Gonçalves')
('Brazil', 'Luís', 'Gonçalves')
('Brazil', 'Luís', 'Gonçalves')
('Germany', 'Leonie', 'Köhler')
('Germany', 'Leonie', 'Köhler')
('Germany', 'Leonie', 'Köhler')
SELECT i."BillingCountry", c."FirstName", c."LastName" 
FROM invoices AS i, customers AS c 
WHERE i."CustomerId" = c."CustomerId"
 LIMIT :param_1
